In [7]:
import json
import boto3
import asyncio
import argparse
from boto3.session import Session

In [8]:
async def invoke_agent(question):
    boto_session = Session()
    region_name = boto3.Session().region_name


    #SSM을 활용해서 Bedrock AgenticCore ARN 가져오기
    ssm_client = boto_session.client('ssm', region_name=region_name)
    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_agentic_core/runtime_iam/agent_arn"
    )
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region_name)

    
    #Streaming으로 Bedrock AgenticCore Application 호출하기
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(
        None,
        lambda: agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn_response["Parameter"]["Value"],
            qualifier="DEFAULT",
            payload=json.dumps({"input_data": question})
        )
    )

    #응답을 객체로 받은뒤 계속적으로 오는 streaming 결과 출력
    for line in response['response'].iter_lines():
        if line and line.startswith(b'data: '):
            data = json.loads(line[6:].decode('utf-8'))
            if data['type'] == 'stream':
                print(data['content'], end='')
    print()

In [10]:
await invoke_agent("태양의 온도는 얼마인가요?")


🤖 직접 답변: 태양의 중심부 온도는 약 1,500만도 정도로 매우 높습니다. 하지만 태양 표면의 온도는 약 5,800K(5,500°C) 정도입니다. 

태양은 핵융합 반응을 통해 엄청난 에너지를 방출하는 별입니다. 중심부에서 수소 원자핵이 융합하면서 헬륨으로 바뀌는 과정에서 막대한 열에너지가 발생합니다. 

이렇게 발생한 열에너지가 태양 내부에서 복사와 대류 과정을 거쳐 표면까지 전달되면서 표면 온도가 결정됩니다. 표면에서는 이 열에너지가 가시광선, 적외선, 자외선 등의 전자기파 형태로 우주 공간으로 방출됩니다.


In [11]:
await invoke_agent("3345533*3을 알려주세요")


🔧 도구 'multiply_numbers' 결과: meta=None content=[TextContent(type='text', text='10036599', annotations=None, meta=None)] structuredContent={'result': 10036599} isError=False


In [12]:
await invoke_agent("3345533+3을 알려주세요")


🔧 도구 'add_numbers' 결과: meta=None content=[TextContent(type='text', text='3345536', annotations=None, meta=None)] structuredContent={'result': 3345536} isError=False
